### Programación Concurrente
## 21. Introducción a Spark

Spark es la herramienta, dentro del ecosistema de Hadoop, que permite trabajar con Cómputo Paralelo y Distribuido.

Para ello, un ambiente de Spark necesita contar con los siguientes elementos.


![Hilos](https://d39kqat1wpn1o5.cloudfront.net/app/uploads/2019/10/spark-architecture-1024x542.png)

- El Master Node contiene el Drive: distribuye y asigna tareas a los Workers.
- Los Worker Nodes ejecutan las tareas.

(Nota: estos elementos son Procesos al fin y al cabo, entonces pueden ser ejecutados en una sola computadora, si trabajamos de manera local).

- El Cluster Manager administra los recursos, para que las tareas puedan ser completados.



### Particiones

Para poder paralelizar tareas, nuestros DataFrames de Spark se guardan con particiones: es decir, un grupo de hileras serán procesados un Worker nodo, otras por otro nodo, etc. Las particiones normalmente se crean en automático.

### Lazy Evaluation

Además, Spark no ejecuta las tareas de inmediato. En realidad, se espera a tener un grupo de tareas y se espera hasta que estas realmente tengan que ser ejecutadas (por ejemplo, al mandar a guardar un output), para empezar el procesamiento.

Esto lo hace porque se espera a tener un set de instrucciones y así poder optimizarlas.

### Iniciar un programa con Spark

Para empezar a trabajar con Spark, necesitamos forzosamente crear una SparkSession. Además, si no lo hemos hecho ya, es necesario installar e importar la librería `pyspark`.



In [1]:
!pip install pyspark==3.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317123 sha256=8179f26b7639dac9eaa0c799a8cfb5ba68acce53dd081a880dc16938b887c70b
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.3
    Uninstalling pyspark-3.5.3:
      Successfully uninstalled pyspark-3.5.3


In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

Vamos a trabajar con datos del Covid-19 en México. Revísalos y descárgalos de la siguiente página web:

https://www.kaggle.com/datasets/tavoglc/covid19-data-from-mexico?resource=download

In [24]:
df = spark.read.csv('covidmex.csv',header=True)
df.show(5)

+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+
|ID_REGISTRO|ENTIDAD_RES|MUNICIPIO_RES|FECHA_INGRESO|FECHA_SINTOMAS|covidt|delta|               lat|               long|               alt|                 qry|dayofyear|       lengthofday|
+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+
|     z526b3|          9|           12|   2020-12-21|    2020-12-18|     1|    3| 19.20155345588235| -99.20180252450989|3008.9460784313724|Cve_Ent==9 & Cve_...|      356|11.768371962176587|
|     z3d1e2|          9|            5|   2020-04-22|    2020-04-20|     1|    2|         19.482945|         -99.113471|            2229.0|Cve_Ent==9 & Cve_...|      113|13.441805501598786|
|     z21f6f|          7|            9|   2020-04-

### Análisis exploratorio usando SparkDataFrames

Manipular datos y hacer cálculos a veces es similar, pero normalmente es distinto a como lo haríamos en Pandas. De hecho, ya viste una diferencia: el uso del método `.show(n)`

Otras diferencias son:

In [12]:
# Conocer la dimension de mi DataFrame

print((df.count(), len(df.columns)))

(5723141, 13)


In [13]:
# ¿Qué tipos de datos tengo?

df.printSchema()

root
 |-- ID_REGISTRO: string (nullable = true)
 |-- ENTIDAD_RES: string (nullable = true)
 |-- MUNICIPIO_RES: string (nullable = true)
 |-- FECHA_INGRESO: string (nullable = true)
 |-- FECHA_SINTOMAS: string (nullable = true)
 |-- covidt: string (nullable = true)
 |-- delta: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- alt: string (nullable = true)
 |-- qry: string (nullable = true)
 |-- dayofyear: string (nullable = true)
 |-- lengthofday: string (nullable = true)



### Tarea

Aplica los cambios siguientes:
1. Modifica tu DataFrame: agrega una columna nueva llamada "PAIS", en donde todas las hileras tendrán el valor "Mexico".

2. Elimina la columna *lengthofday* de tu DataFrame.

3. Crea un nuevo DataFrame, llamado `ubicacion`, que tendrá solamente los valores de *lat*, *lon*, *ENTIDAD_RES* y *PAIS* de `df`.

4. Filtra la columna *ENTIDAD_RES* dentro de `ubicacion` para quedarte con solo los valores = 9 (Ciudad de México). Guarda el resultado en un DataFrame llamado `cdmx`

5. Selecciona los primeros 5 valores de `cdmx` y guárdalos como un DataFrame de Pandas, con el nombre *df_pandas*.

####Tip
Métodos y Funciones que podrían servirte:

- .select()
- .drop()
- .withcolumn()
- lit()
- .filter()
- .limit()
- toPandas()



## 1

In [25]:
from pyspark.sql.functions import lit

df = df.withColumn("PAIS", lit("Mexico"))
df.show(5)


+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+------+
|ID_REGISTRO|ENTIDAD_RES|MUNICIPIO_RES|FECHA_INGRESO|FECHA_SINTOMAS|covidt|delta|               lat|               long|               alt|                 qry|dayofyear|       lengthofday|  PAIS|
+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------------------+------+
|     z526b3|          9|           12|   2020-12-21|    2020-12-18|     1|    3| 19.20155345588235| -99.20180252450989|3008.9460784313724|Cve_Ent==9 & Cve_...|      356|11.768371962176587|Mexico|
|     z3d1e2|          9|            5|   2020-04-22|    2020-04-20|     1|    2|         19.482945|         -99.113471|            2229.0|Cve_Ent==9 & Cve_...|      113|13.441805501598786|Mexico|
|     z21f6f|  

## 2

In [26]:
df=df.drop("lengthofday")
df.show(5)

+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------+
|ID_REGISTRO|ENTIDAD_RES|MUNICIPIO_RES|FECHA_INGRESO|FECHA_SINTOMAS|covidt|delta|               lat|               long|               alt|                 qry|dayofyear|  PAIS|
+-----------+-----------+-------------+-------------+--------------+------+-----+------------------+-------------------+------------------+--------------------+---------+------+
|     z526b3|          9|           12|   2020-12-21|    2020-12-18|     1|    3| 19.20155345588235| -99.20180252450989|3008.9460784313724|Cve_Ent==9 & Cve_...|      356|Mexico|
|     z3d1e2|          9|            5|   2020-04-22|    2020-04-20|     1|    2|         19.482945|         -99.113471|            2229.0|Cve_Ent==9 & Cve_...|      113|Mexico|
|     z21f6f|          7|            9|   2020-04-27|    2020-04-24|     1|    3| 16.21599257359306| -93.93758

## 3

In [28]:
ubicacion = df.select("lat", "long", "ENTIDAD_RES", "PAIS")
ubicacion.show(5)

+------------------+-------------------+-----------+------+
|               lat|               long|ENTIDAD_RES|  PAIS|
+------------------+-------------------+-----------+------+
| 19.20155345588235| -99.20180252450989|          9|Mexico|
|         19.482945|         -99.113471|          9|Mexico|
| 16.21599257359306| -93.93758461688306|          7|Mexico|
| 26.06980561012655|-100.27776220506335|         19|Mexico|
|25.679631916666665|-100.16954308333334|         19|Mexico|
+------------------+-------------------+-----------+------+
only showing top 5 rows



## 4

In [29]:
cdmx=ubicacion.filter(ubicacion.ENTIDAD_RES == 9)
cdmx.show(5)


+-----------------+------------------+-----------+------+
|              lat|              long|ENTIDAD_RES|  PAIS|
+-----------------+------------------+-----------+------+
|19.20155345588235|-99.20180252450989|          9|Mexico|
|        19.482945|        -99.113471|          9|Mexico|
|        19.359004|        -99.092623|          9|Mexico|
|        19.395901|        -99.097613|          9|Mexico|
|19.20155345588235|-99.20180252450989|          9|Mexico|
+-----------------+------------------+-----------+------+
only showing top 5 rows



## 5

In [30]:
df_pandas=cdmx.limit(5).toPandas()
df_pandas

,lat,long,ENTIDAD_RES,PAIS
0,19.20155345588235,-99.20180252450989,9,Mexico
1,19.482945,-99.113471,9,Mexico
2,19.359004,-99.092623,9,Mexico
3,19.395901,-99.097613,9,Mexico
4,19.20155345588235,-99.20180252450989,9,Mexico
